In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from scipy import stats

In [4]:
from ast import literal_eval

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [6]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [7]:
from nltk.stem.snowball import SnowballStemmer

In [8]:
from nltk.stem.wordnet import WordNetLemmatizer

In [9]:
from nltk.corpus import wordnet

In [11]:
from surprise import Reader, Dataset, SVD

In [12]:
from surprise.model_selection import cross_validate

In [13]:
import warnings; warnings.simplefilter('ignore')

In [14]:
md=pd.read_csv('C:/Users/Miti/Documents/movierecommendedsystems datasets/movies_metadata.csv')

In [15]:
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [16]:
md['genres']=md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])

In [34]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages=md[md['vote_average'].notnull()]['vote_average'].astype('int')
C=vote_averages.mean()

In [35]:
C

5.244896612406511

In [36]:
m = vote_counts.quantile(0.95)

In [37]:
m

434.0

In [38]:
md['year']=pd.to_datetime(md['release_date'],errors='coerce').apply(lambda x: str(x).split('-')[0] if x !=np.nan else np.nan)

In [39]:
qualified=md[(md['vote_count']>=m)&(md['vote_count'].notnull())&(md['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity','genres']]

In [40]:
qualified['vote_count']=qualified['vote_count'].astype('int')

In [41]:
qualified['vote_average']=qualified['vote_count'].astype('int')

In [42]:
qualified.shape

(2274, 6)

In [43]:
def weighted_rating(x):
    v=x['vote_count']
    R=x['vote_average']
    return (v/(v+m)*R)+(m/(m+v)* C)

In [44]:
qualified['wr']=qualified.apply(weighted_rating, axis=1)

In [45]:
qualified=qualified.sort_values('wr',ascending=False).head(250)

In [46]:
qualified.head()

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,14075,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",13654.138899
12481,The Dark Knight,2008,12269,12269,123.167,"[Drama, Action, Crime, Thriller]",11850.006871
14551,Avatar,2009,12114,12114,185.071,"[Action, Adventure, Fantasy, Science Fiction]",11695.192245
17818,The Avengers,2012,12000,12000,89.8876,"[Science Fiction, Action, Adventure]",11581.331533
26564,Deadpool,2016,11444,11444,187.86,"[Action, Adventure, Comedy]",11026.049191


In [47]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [48]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [49]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,7.744878
19901,Paperman,2012,734,8,7.19863,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.9943,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


In [50]:
#content based movie recommendation system
links_small = pd.read_csv('C:/Users/Drishya/Documents/movierecommendedsystems datasets/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [51]:
md = md.drop([19730, 29503, 35587])

In [52]:
md['id'] = md['id'].astype('int')

In [53]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [54]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [55]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [56]:
tfidf_matrix.shape

(9099, 268124)

In [57]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [58]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [59]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [60]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [61]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [62]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [64]:
credits = pd.read_csv('C:/Users/Drishya/Documents/movierecommendedsystems datasets/credits.csv')
keywords = pd.read_csv('C:/Users/Drishya/Documents/movierecommendedsystems datasets/keywords.csv')

In [65]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')
md.shape
(45463, 25)
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [66]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))


In [67]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [69]:
md['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [70]:
s = s[s > 1]
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')
'dog'
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [71]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [72]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
1134                Batman Returns
7659    Batman: Under the Red Hood
Name: title, dtype: object

In [73]:
get_recommendations('Mean Girls').head(10)


3319                         Head Over Heels
4763                           Freaky Friday
1329                        The House of Yes
6277                        Just Like Heaven
7905                   Mr. Popper's Penguins
7332              Ghosts of Girlfriends Past
6959               The Spiderwick Chronicles
8883                                The DUFF
5152    Confessions of a Teenage Drama Queen
6698                   It's a Boy Girl Thing
Name: title, dtype: object

In [74]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified
improved_recommendations('The Dark Knight')


,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
524,Batman,2145,7,1989,6.704647
6622,Children of Men,2120,7,2006,6.701756
8419,Man of Steel,6462,6,2013,5.952478
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943


In [75]:
improved_recommendations('Mean Girls')

,title,vote_count,vote_average,year,wr
8477,About Time,2140,7,2013,6.704073
8883,The DUFF,1372,6,2015,5.818541
3712,The Princess Diaries,1063,6,2001,5.781086
4763,Freaky Friday,919,6,2003,5.757786
1637,The Parent Trap,768,6,1998,5.727359
6277,Just Like Heaven,595,6,2005,5.681521
6959,The Spiderwick Chronicles,593,6,2008,5.680901
8989,Aloha,704,5,2015,5.093396
7332,Ghosts of Girlfriends Past,716,5,2009,5.092422
7905,Mr. Popper's Penguins,775,5,2011,5.087912


In [76]:
reader = Reader()
ratings = pd.read_csv('C:/Users/Drishya/Documents/movierecommendedsystems datasets/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [87]:
from surprise.model_selection import cross_validate, KFold

In [90]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)


AttributeError: 'DatasetAutoFolds' object has no attribute 'split'

In [91]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

NameError: name 'evaluate' is not defined

In [93]:
trainset=data.build_full_trainset()
svd.fit(trainset)

In [94]:
ratings[ratings['userId']==1]


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [95]:
svd.predict(1,302,3)

Prediction(uid=1, iid=302, r_ui=3, est=2.6612381967285863, details={'was_impossible': False})

In [96]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan
id_map = pd.read_csv('C:/Users/Drishya/Documents/movierecommendedsystems datasets/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [97]:
indices_map = id_map.set_index('id')
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [98]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
987,Alien,4564.0,7.9,1979,348,3.327529
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.235538
974,Aliens,3282.0,7.7,1986,679,3.218179
8712,Guardians of the Galaxy,10014.0,7.9,2014,118340,3.139834
1011,The Terminator,4208.0,7.4,1984,218,3.128304
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.985175
344,True Lies,1138.0,6.8,1994,36955,2.914642
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.868001
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.838375
922,The Abyss,822.0,7.1,1989,2756,2.830713


In [99]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
987,Alien,4564.0,7.9,1979,348,3.533136
974,Aliens,3282.0,7.7,1986,679,3.340193
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.298518
8712,Guardians of the Galaxy,10014.0,7.9,2014,118340,3.245111
4966,Hercules in New York,63.0,3.7,1969,5227,3.156875
1011,The Terminator,4208.0,7.4,1984,218,3.150394
922,The Abyss,822.0,7.1,1989,2756,3.116465
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.030424
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.996859
8419,Man of Steel,6462.0,6.5,2013,49521,2.949037
